In [1]:

import warnings
warnings.filterwarnings("ignore")
from tkinter import *
import tkinter.ttk as ttk
from PIL import Image,ImageTk
import os
import glob
from tkinter import filedialog
import pandas as pd
import numpy as np
import keras
import cv2
import csv
import tensorflow as tf
from tkinter.filedialog import asksaveasfile
from keras.preprocessing.image import img_to_array,load_img
from tensorflow.python.keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 

print(tf.__version__)

Using TensorFlow backend.


2.3.1


In [2]:
image_seg=tf.keras.models.load_model(r"Image_Seg.h5")

In [3]:
df=None
def csv():
    global path,files,df
    inp_images = []
    res_images = []
    for f1 in files:
        inp_images.append(Image.open(path+"/"+f1).resize((200,200),Image.ANTIALIAS))
        image = test_image(path+"/"+f1)
        res_images.append(image)
    display(inp_images,res_images)

In [4]:
def test_image(path):    
    image = read_image(path)
    image = np.expand_dims(image, axis=0)
    res_image=image_seg.predict(image)
    #print(res_image.shape)
    #print(res_image[:,:,:,1][0].shape)
    data = res_image[:,:,:,1][0]
    rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)
    rescaled = np.repeat(rescaled[:,:,np.newaxis], 3, axis=2)
    image = Image.fromarray(rescaled, 'RGB').resize((200,200))
    return image

In [5]:
path=''
files=[]
IMG_SIZE=256

In [6]:
def res():
    global path,cnn,dnn,l5,l6,data
    image = test_image(path)
    photo = ImageTk.PhotoImage(image)
    l5.config(image=photo)
    l5.image = photo
    l5.grid(row = 2,column = 2, pady = 2)       

In [7]:
def sel():
    global b1,l1,b2,l2,l4,l5,TableMargin,rb1,rb2,f2
    f2.grid(row=1)
    b1.grid(row = 0, column = 0,pady = 2)
    if v.get() == 1:
        rb1.config(bg='magenta')
        rb2.config(bg = "light blue")
        b1.config(text="Browse your folder",command=getDirectory)
        forgetlabels(l1,b2,l2,l4,l5)
    else:
        rb1.config(bg = "light blue")
        rb2.config(bg='magenta')
        b1.config(text="Browse your image",command=getImage)
        forgetlabels(l1,b2,l2,l4,l5)
def forgetlabels(l1,b2,l2,l4,l5):
    l1.grid_remove()
    b2.grid_remove()
    l2.grid_remove()
    l4.grid_remove()
    lpic.grid_remove()
    l5.grid_remove()
    l6.grid_remove()
    f3.grid_remove()
    f5.destroy()
def close():
    root.destroy()
def read_image(file_path):
    image = load_img(file_path, target_size=(256,256)) 
    image = img_to_array(image) 
    image /= 255. 
    return image
def getImage():
    global l1,l2,l3,l4,b2,lpic,path,f3
    f3.grid(row=2)
    path = filedialog.askopenfilename(title='Select an Image')
    l1.config(text="PATH : "+path)
    image = Image.open(path).resize((200,200),Image.ANTIALIAS)
    photo = ImageTk.PhotoImage(image)
    #print(type(photo))
    lpic.config(image=photo)
    lpic.image = photo
    l1.grid(row = 0, column = 1, pady = 2)
    b2.config(command=res)
    b2.grid(row = 1, column = 1, pady = 2,sticky='w')
    l2.grid(row = 1, column = 0, pady = 2)
    #l3.grid(row = 1, column = 1, pady = 2)
    l4.grid(row = 1, column = 2, pady = 2)
    lpic.grid(row = 2,column = 0, pady = 2)
def getDirectory():
    global path,files,l1
    path = filedialog.askdirectory()
    l1.config(text="PATH : "+path)
    l1.grid(row = 0, column = 1, pady = 2)
    files = os.listdir(path)
    path = path.replace("\\","/")
    b2.config(command=csv)
    b2.grid(row = 1, column = 1, pady = 2,sticky='w')

In [8]:
def display(images,result): 
    global f5
    f5 = Frame(root)
    f5.grid(row=3)
    canvas = Canvas(f5,width= 410)
    scrollbar = ttk.Scrollbar(f5, orient="vertical", command=canvas.yview)
    scrollable_frame = Frame(canvas)
    scrollable_frame.bind("<Configure>",lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)
    f6 = Frame(f5)
    f6.pack(side="top")
    Label(f6,text = "Image",width=15,fg='white',bg='light blue',font=('arial',16,'bold'),borderwidth=3, relief="ridge").grid(row = 0, column = 0, pady = 2)
    Label(f6).grid(row=0,column=1)
    Label(f6,text = "Segmented Image",width=15,fg='white',bg='light blue',font=('arial',16,'bold'),borderwidth=3, relief="ridge").grid(row = 0, column = 2, pady = 2)
    for i in range(len(images)):
        photo = ImageTk.PhotoImage(images[i])    
        pic = ImageTk.PhotoImage(result[i])
        label_name = 'l'+str(i)
        label_name = Label(scrollable_frame, image = photo,bg='orange')
        label_name.image = photo
        label_name.grid(row = i, column = 0, pady = 2)
        Label(scrollable_frame).grid(row = i, column = 1, pady = 2)
        label_name = 'll'+str(i)
        label_name = Label(scrollable_frame, image = pic,bg='orange')
        label_name.image = pic
        label_name.grid(row = i, column = 2, pady = 2)
        #print(i)
    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

In [ ]:
root = Tk()
root.geometry("800x600")
root.title("Image Segmentation for detecting infected tissue")
v = IntVar()

f1 = Frame(root, width=100, height =100)
f1.grid(row=0)

f2 = Frame(root, width=100, height =50)

f3 = Frame(root)
#f3.pack(side = TOP)

f4 = Frame(root)
f4.grid(row=5)

f5 = Frame(root)

Label(f1).grid(row=0)
logo_image = ImageTk.PhotoImage(Image.open(r"sastra_logo.jpeg").resize((220, 40), Image.ANTIALIAS))
logo = Label(f1, image = logo_image,bd=1,padx=2,pady=2,bg='red',borderwidth=3, relief="flat", highlightcolor="white",anchor='w')
logo.grid(row=1,column=0)
title = Label(f1,text="Image Segmentation",width=20,fg='white',bg='purple',font=('arial',24,'bold'),borderwidth=3, relief="ridge").grid(row = 1, column = 1, pady = 2)
Label(f1).grid(row=2,pady=0.5)
label = Label(f1,text="""Choose an input option:""",width=20,justify = LEFT,fg='black',bg='white',font=('arial',12,'bold'), relief="ridge",anchor='w')
label.grid(row = 3, column = 0, pady = 2)
rb1 = Radiobutton(f1,text="Group of images",variable=v,value=1,command=sel,background = "light blue",relief='groove')
rb1.grid(row = 3, column = 1)
rb2 = Radiobutton(f1,text="A single image",variable=v,value=2,command=sel,background = "light blue",relief='groove')
rb2.grid(row = 3, column = 2,sticky='w')

b1 = Button(f2,bg='white',width=22,relief=RIDGE,font=('arial',12,'italic')) 
l1 = Label(f2,fg='black',bg='white',font=('arial',12,'bold'))
b2 = Button(f2,text='DETECT',bg='yellow',width=22,relief=RIDGE,font=('arial',12,'italic'))  
l2 = Label(f3,text="Input Image",width=20,justify = LEFT,fg='black',bg='white',font=('arial',12,'bold'), relief="ridge")
l4 = Label(f3,text="Predicted Image",width=15,justify = LEFT,fg='black',bg='white',font=('arial',12,'bold'), relief="ridge")
lpic = Label(f3,bg='red')
l5 = Label(f3,bg='red')
l6 = Label(f3,fg='black',bg='white',font=('arial',12,'bold'), relief="ridge",pady=25)
Label(f4).pack(side="top")
Button(f4,text='EXIT',bg='red',width=22,relief=RIDGE,font=('arial',12,'italic'),command=close).pack(side="top")
root.mainloop()